![](../header.jpg)

# Reference Frames

Kevin Walchko

21 May 2022

---

Using local tangental frames to the Earth's surface is useful for navigational purposes, especially in robotics. There are two common frames used, NED and ENU. Below we will convert between geodetic frame (GPS), ECEF (global) and the local frames.

In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
from ins_nav import NavigationFrame
from ins_nav import WGS84
import numpy as np
from numpy.linalg import norm
np.set_printoptions(precision=4, suppress=True)

rad2deg = 180 / np.pi

In [27]:
ref = (40,-90, 0) # (lat, lon, altitude)

wgs = WGS84()
frame = NavigationFrame(ref)

In [12]:
print(f"Frame origin ll: {frame.latr*rad2deg:.2f}, {frame.lonr*rad2deg:.2f}")
print(f"Frame origin ECEF: {frame.ecefr}")

Frame origin ll: 40.00, -90.00
Frame origin ECEF: [       0.     -4892707.6001  4077985.5722]


In [16]:
# convert origin to NED ... should be [0,0,0]
refecef = wgs.llh2ecef(ref)
frame.ecef2ned(refecef)

array([-0., -0.,  0.])

In [21]:
# let's say we move 100 m in NED frame
pos = [100,0,0]
posecef = frame.ned2ecef(pos)
print(posecef)

[       0.     -4892643.3213  4078062.1766]


In [22]:
# lets see if we have moved 100m in ECEF frame too? should be "YES"
print(f"Change in distance: {norm(posecef - frame.ecefr)} m")

Change in distance: 100.00000000006042


In [24]:
# going to do this the long way first
#
# NED -> ECEF -> ENU
ned = [500, 600, -400]
tmp = frame.ned2ecef(ned)
enu = frame.ecef2enu(tmp)

print(f"NED: {ned}")
print(f"ENU: {enu}")

NED: [500, 600, -400]
ENU: [600. 500. 400.]


In [25]:
# Also notice this matrix is the same when transposed!
#
# Rned2enu == Rned2enu.T
#
# Thus it could be used for both conversions without transpose
Rned2enu = np.array([
    [0,1,0],
    [1,0,0],
    [0,0,-1]
])

In [26]:
# this should give the originals above
print(f"Now using the matrix, ENU: {Rned2enu @ ned}")
print(f"And back to NED: {Rned2enu.T @ enu}")

Now using the matrix, ENU: [600 500 400]
And back to NED: [ 500.  600. -400.]


In [28]:
# we can also do this with NaviagtionalFrame
# this should give the originals above
print(f"Now using the matrix, ENU: {frame.ned2enu(ned)}")
print(f"And back to NED: {frame.enu2ned(enu)}")

Now using the matrix, ENU: [600 500 400]
And back to NED: [ 500.  600. -400.]
